In [ ]:
'''
import gensim.downloader as api
import pandas as pd

wv = api.load('word2vec-google-news-300')

df = pd.read_csv('qqpTrain.csv')

data = df.to_dict(orient='records')

data = data[:3500]
newData = []
bm25Query = []
bm25Answer = []
for idx1,d1 in enumerate(data):
    bm25Query.append(d1["question1"])
    bm25Answer.append(d1["question2"])
    for idx2,d2 in enumerate(data):
        isDuplicate = 0
        if idx1 == idx2 and d1["is_duplicate"] == 1:
            isDuplicate = 1
        newData.append({"question1":d1['question1'],"question2":d2['question2'],"is_duplicate":isDuplicate})
data = newData
'''


In [ ]:
''' #build dataset
import os
import csv
dataset_path = 'quora-dataset/'
first5000 = []
#counter = 0
with open(os.path.join(dataset_path, 'classification', 'test_pairs.tsv'), 'r', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        
       
        #if len(first5000) >= 5000:
        #    break
        first5000.append((row['question1'], row['question2'], row['is_duplicate']))
        
'''        
'''
pairs = []
for Iidx in range(len(first5000)):
    for Jidx in range(len(first5000)):
        label = 0
        if Iidx == Jidx and first5000[2] == 1:
            label = 1
        pairs.append((first5000[Iidx][0], first5000[Jidx][1], label))
'''s

In [ ]:
'''
import gensim.downloader as api
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from scipy import spatial

tokenizer = RegexpTokenizer(r'\w+')


wv = api.load('word2vec-google-news-300')

w2vResult = []
invalidList = set() #sync data between w2v and bm25
try:
    for idx,d in enumerate(first5000):
        if type(d[0]) != str or type(d[1]) != str:
            invalidList.add(idx)
            continue

        query = []
        answer = []
        for q in tokenizer.tokenize(d[0]):
            if q in wv:
                query.append(wv[q])
        for a in tokenizer.tokenize(d[1]):
            if a in wv:
                answer.append(wv[a])
        if not query or not answer:
            invalidList.add(idx)
            continue
        query = np.max(np.array(query),0)
        answer = np.max(np.array(answer),0)

        doc_scores = 1 - spatial.distance.cosine(query, answer)
        w2vResult.append((doc_scores, int(d[2])))
        
except Exception as e:
    print(e)


import pickle

# Define the file path
file_path = "qqpTestw2v.pkl"

# Serialize and save the list to a binary file using pickle
with open(file_path, "wb") as file:
    pickle.dump(w2vResult, file)
#'''


In [ ]:
from rank_bm25 import BM25Plus

bm25Result = []
for r in first5000:

    tokenized_answer = [r[0].split(" ")]
    tokenized_question = r[1].split(" ")
    bm25 = BM25Plus(tokenized_answer)
    doc_scores = bm25.get_scores(q)
    bm25Result.append((doc_scores[0], int(r[2])))


file_path = "qqpTestbm25P.pkl"

# Serialize and save the list to a binary file using pickle
with open(file_path, "wb") as file:
    pickle.dump(bm25Result, file)


In [ ]:
'''  # all data, not diag
import gensim.downloader as api
import pandas as pd

wv = api.load('word2vec-google-news-300')

w2vResult = []
invalidList = set() #sync data between w2v and bm25
try:
    for idx,d in enumerate(data):
        if type(d['question1']) != str or type(d['question2']) != str:
            invalidList.add(idx)
            continue

        query = []
        answer = []
        for q in tokenizer.tokenize(d['question1']):
            if q in wv:
                query.append(wv[q])
        for a in tokenizer.tokenize(d['question2']):
            if a in wv:
                answer.append(wv[a])
        if not query or not answer:
            invalidList.add(idx)
            continue
        query = np.max(np.array(query),0)
        answer = np.max(np.array(answer),0)

        doc_scores = 1 - spatial.distance.cosine(query, answer)
        w2vResult.append((doc_scores, d['is_duplicate']))
        if d['is_duplicate'] == 1:
            print(doc_scores)
except Exception as e:
    print(e)


import pickle

# Define the file path
file_path = "qqp5000w2v.pkl"

# Serialize and save the list to a binary file using pickle
with open(file_path, "wb") as file:
    pickle.dump(w2vResult, file)
'''

In [ ]:
''' # fix old labels
newbm25Result = []
neww2vResult = []
newBertResult = []
for idx in range(len(labels)):
    newbm25Result.append((bm25Result[idx][0],labels[idx]))
    neww2vResult.append((w2vResult[idx][0], labels[idx]))
    newBertResult.append((bertResult[idx][0], labels[idx]))

import pickle

# Define the file path
file_path = "qqp5000bm25P.pkl"

# Serialize and save the list to a binary file using pickle
with open(file_path, "wb") as file:
    pickle.dump(newbm25Result, file)


# Define the file path
file_path = "qqp5000w2v.pkl"

# Serialize and save the list to a binary file using pickle
with open(file_path, "wb") as file:
    pickle.dump(neww2vResult, file)


    # Define the file path
file_path = "qqp500025xBert.pkl"

# Serialize and save the list to a binary file using pickle
with open(file_path, "wb") as file:
    pickle.dump(newBertResult, file)
'''

In [ ]:
'''#varification of inaccuracy
iidx = 0
jidx = 0
counter = 0
truePos = 0
for idx, r in enumerate(fullBertResult):
    if r[0] > 0.9 and r[1] == 0:
        counter += 1
        if counter == 2893:

            iidx = idx//5000
            jidx = idx%5000
            print(r[0])
            print(idx)
            break
    if r[0] > 0.9 and r[1] == 1:
        truePos += 1
print(counter)
print(truePos)
print(first5000[iidx][0])
print(first5000[jidx][1])

from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np
#import os
#import csv
from sentence_transformers import InputExample
#import pickle
model = CrossEncoder('./output/training_quora-2023-09-28_16-05-03')
print(model.predict([first5000[iidx][0],first5000[jidx][1]]))
'''